In [1]:
"""
コーパスの確認をするスクリプトです。
"""
import re
import pandas as pd

clean_text = re.compile('[!"#$＄%&\'\\\\()*+,-./:：;<=>?@[\\]^_`{|}~「」｣〔〕＜＞～“”〈〉『』【】＆＊✴・（）↑＄＃＠Ⓣ。、？…！｀＋￥％⚫︎⭕〇◯△◇☆✩☆✧･　※ ◎○▼▲■◆♦●★♡┗└┌✿ё◉♪♫♬]')

data = pd.read_csv("../data/train_data/cookpad_steps.csv",names=["id","step"])
cleaned_data = data.apply(lambda data:data.str.replace(clean_text,'',regex=True))
cleaned_data.head()

,id,step
0,ad7d585b06850f8437ff5fb97d3c7a823ff21bb1,鍋に豚肉をいれて酒砂糖みりん醤油しょうが薄切りにんにくで煮込みます
1,4afce5687dc173ad4fef943b686582a1cd06e264,にんじんとれんこんをおろし金でおろします
2,4afce5687dc173ad4fef943b686582a1cd06e264,挽肉と玉ねぎのみじん切りを加えよく
3,混ぜます,NaN
4,4afce5687dc173ad4fef943b686582a1cd06e264,塩コショウを加えピーマンに詰めあとは焼くだけ


In [3]:
cleaned_data = cleaned_data.dropna(subset=["step"])
cleaned_data["str_len"] = cleaned_data["step"].apply(lambda data:len(data))
output = cleaned_data[cleaned_data["str_len"] != 0]
output.head()

/tmp/ipykernel_17539/2115145791.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  cleaned_data["str_len"] = cleaned_data["step"].apply(lambda data:len(data))


,id,step,str_len
0,ad7d585b06850f8437ff5fb97d3c7a823ff21bb1,鍋に豚肉をいれて酒砂糖みりん醤油しょうが薄切りにんにくで煮込みます,33
1,4afce5687dc173ad4fef943b686582a1cd06e264,にんじんとれんこんをおろし金でおろします,20
2,4afce5687dc173ad4fef943b686582a1cd06e264,挽肉と玉ねぎのみじん切りを加えよく,17
4,4afce5687dc173ad4fef943b686582a1cd06e264,塩コショウを加えピーマンに詰めあとは焼くだけ,22
5,4afce5687dc173ad4fef943b686582a1cd06e264,少し蒸らして火を通しできあがり,15


In [5]:
"""
コーパスの作成
"""
from tqdm import tqdm as tq
import MeCab

m = MeCab.Tagger("-d /usr/lib/x86_64-linux-gnu/mecab/dic/mecab-ipadic-neologd -Owakati")

# 合計語彙数
num = 0
# データの総文字数
str_len = output["str_len"].sum()
# 学習データ
corpas = ""
# 直前のid
id_before = ""

for step in tq(output.itertuples(),total=len(output)):
    if id_before != step.id:
        corpas += "\n"
    id_before = step.id
    wakati = m.parse(step.step)
    wakati = wakati.replace("\n","")
    num += len(wakati.split(' '))
    corpas += wakati

with open("../data/train_data/cookpad_step_corpas_GROUPED_BY_ID.txt",'w') as out_corpas:
    out_corpas.write(corpas)

print("平均語彙サイズ:",str_len/num)
print("平均語彙数:",num/len(output))

100%|██████████| 8702111/8702111 [07:30<00:00, 19328.58it/s]


平均語彙サイズ: 1.8377319978969409
平均語彙数: 18.076336649808304
